In [1]:
from libs.alibaba.alibaba import Alibaba
from libs.alibaba.inquiry import Inquiry
from libs.alibaba.inquiry import element_has_css_class
from libs.email import Email
from libs.json import JSON

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

from string import Template
from datetime import datetime
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

import json
import requests
import time
import re
import math

import types
import traceback
import threading

import pendulum

import logging
from logging.handlers import TimedRotatingFileHandler

In [2]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
market

. storage markets.json
.\storage\markets.json was deserialized!


{'name': 'Eyelashes',
 'directory': 'E:\\glitter\\Eyelashes',
 'lid': 'eyelash@glittergroupcn.com',
 'lpwd': 'Wangzhen68970747',
 'homepage': 'https://glittereyelash.en.alibaba.com',
 'lname': 'Jeremy Wang',
 'mobile': '17685593100',
 'accounts': [{'lid': 'eyelash1@glittergroupcn.com',
   'lpwd': 'Zhangying25160',
   'lname': 'Emily Zhang',
   'mobile': '17685593100',
   'disabled': False},
  {'lid': 'eyelash2@glittergroupcn.com',
   'lpwd': 'Huyinping18130',
   'lname': 'Ada Alva',
   'mobile': '18563918130',
   'disabled': False}]}

In [6]:
account = market['accounts'][1]
print(account)
inquiry = Inquiry(market, account, headless=False)

{'lid': 'eyelash2@glittergroupcn.com', 'lpwd': 'Huyinping18130', 'lname': 'Ada Alva', 'mobile': '18563918130', 'disabled': False}
E:\glitter\Eyelashes_config [] inquiry_tracking_ids_Ada.json
E:\glitter\Eyelashes_config\inquiry_tracking_ids_Ada.json was deserialized!


In [4]:
inquiry = Inquiry(market, headless=False)

E:\glitter\Eyelashes_config [] inquiry_tracking_ids_Jeremy.json
E:\glitter\Eyelashes_config\inquiry_tracking_ids_Jeremy.json was deserialized!


In [ ]:
inquiry.login()

In [7]:
inquiry.load_url()

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Ada


In [ ]:
inquiry.load_url()
time.sleep(1)
inquiry_list = []
while True:
    inquiries = inquiry.get_inquiries()
    inquiry_list = inquiry_list + inquiries
    time.sleep(1)
    for enquiry in inquiries:
        try:
            inquiry.open_inquiry_in_new_tab(enquiry)
            conversation = inquiry.get_conversation()
            enquiry['conversation'] = conversation
        except Exception as e:
            traceback.print_exc()
        finally:
            inquiry.close_inquiry_and_switch_back()
            
    if inquiry.next_page():
        continue
        
    break
len(inquiry_list)

In [ ]:
inquiries = inquiry.get_inquiries()

In [ ]:
inquiry_list = inquiries

In [ ]:
inquiry_list[0]

In [ ]:
enquiries = []
idx = 0
for enquiry in inquiry_list:
    if '【RFQ】 Request for Quotation' in enquiry['tags']:
        continue
#     if 'prestl bernhard' in enquiry['buyer']:
#         continue
        
    first_message = enquiry['conversation']['messages'][0]
    if first_message['position'] == 'right':
        continue
        
    idx += 1
    comming_datetime = pendulum.parse(first_message['date'], tz='Asia/Shanghai')
    enquiry['comming_datetime'] = comming_datetime
#     print(idx, enquiry['id'], comming_datetime, enquiry['buyer_country'], enquiry['buyer'])
    enquiries.append(enquiry)
len(enquiries)

In [ ]:
enquiries.sort(key=lambda eq:eq['comming_datetime'])

In [ ]:
length = len(enquiries)
count = 0
for idx, enquiry in enumerate(enquiries):
    enquiry['duplicated'] = False
    offset = 1
    country = enquiry['buyer_country']
    buyer = enquiry['buyer']
    cdt = enquiry['comming_datetime'].subtract(hours=3)
    while idx-offset >= 0:
        dt = enquiries[idx-offset]['comming_datetime']
        if dt < cdt:
            break
    
        if country == enquiries[idx-offset]['buyer_country'] and buyer == enquiries[idx-offset]['buyer']:
            enquiry['duplicated'] = True

        offset += 1

    if not enquiry['duplicated']:
        count += 1
        
    print(idx, count, enquiry['id'], enquiry['duplicated'], enquiry['comming_datetime'], enquiry['buyer_country'], enquiry['buyer'])


In [ ]:
def search(inquiries, st, et):
    result = []
    for enquiry in inquiries:
        if enquiry['duplicated']:
            continue
        if enquiry['buyer_country'] != 'United States':
            continue
        dt = enquiry['comming_datetime']
        stime = dt.set(hour=int(st.split(':')[0]), minute=int(st.split(':')[1]))
        etime = dt.set(hour=int(et.split(':')[0]), minute=int(et.split(':')[1]))
        if dt >= stime and dt <etime:
#             print(enquiry['comming_datetime'], stime, etime)
            result.append(enquiry)
    return result

In [ ]:
st = '08:00'
et = '09:00'
result = search(enquiries, st, et)
len(result)

In [ ]:
dt = pendulum.parse('15:00:00')
results = []
for i in range(144):
    st = dt.to_time_string()
    dt = dt.add(minutes=10)
    et = dt.to_time_string()
    result = search(enquiries, st, et)
    results.append([i, st, et, result])
    print(len(result), end='\t')

In [ ]:
result = results[60]
print(result[0], result[1], result[2])
for enquiry in result[3]:
    print(enquiry['id'], enquiry['comming_datetime'], enquiry['buyer_country'], enquiry['buyer'])

In [ ]:
count = 0
counts = {}
for enquiry in enquiries:
    if enquiry['duplicated']:
        continue
        
    count += 1
    country = enquiry['buyer_country']
        
    if country not in counts:
        counts[country] = 0
        
    counts[country] += 1
count_list = []
for country in counts:
    count_list.append([country, counts[country]])
count_list.sort(reverse=True, key=lambda x: x[1])

In [ ]:
for item in count_list:
    print(format(item[1]/count*100, '0.2f')+'%', end='\t')
    print(item[0], end='\t')
    print(item[1], end='\t')
    print()

In [ ]:
inquiry.browser.switch_to.default_content()

In [ ]:
inquiry.load_url()

In [ ]:
threads = inquiry.webww_check()

In [ ]:
dialog_iframe = inquiry.browser.find_element_by_css_selector('#webatm2-iframe')
inquiry.browser.switch_to.frame(dialog_iframe)
sidebar = inquiry.browser.find_ele.ment_by_css_selector('.webatm2-sidebar')
head = inquiry.browser.find_element_by_css_selector('.webatm2-window-head')
body = inquiry.browser.find_element_by_css_selector('.webatm2-window-body')


In [ ]:
thread = threads[0]
thread

In [ ]:
inquiry.webww_switch_to_thread(threads[1], sidebar, head, body)

In [ ]:
inquiry.browser.switch_to.default_content()

In [ ]:
for t in threads:
    inquiry.webww_switch_to_thread(t, sidebar, head, body)

In [ ]:
not {}

In [ ]:
inquiry.load_reply_templates()

In [ ]:
inquiry.reply_templates

In [ ]:
def is_auto_reply_needed(self, enquiry):
    eid = enquiry['id']
    #  # for testing purpose
    # if enquiry['id'] == '11947313067':
    #     self.tracking_ids[eid] = {}
    #     self.tracking_ids[eid]['datetime'] = pendulum.now()
    #     self.tracking_ids[eid]['status'] = ['new']
    #     self.save_tracking_ids()
    #     return True
    # else:
    #     return False
    print('==------------------------------------------------------==')
    print(self.lname.lower(), ' - ', enquiry['responsible_person'].lower())
#     if enquiry['responsible_person'].lower() != self.lname.lower():
#         return False

    if eid in self.tracking_ids and self.tracking_ids[eid]['datetime'].diff().in_hours() <= 12:
        if not enquiry['is_replied']:
            return True
    if len(enquiry['tags']) == 0 and enquiry['is_new']:
        if eid not in self.tracking_ids:
            self.tracking_ids[eid] = {}
            self.tracking_ids[eid]['datetime'] = pendulum.now()
            self.tracking_ids[eid]['status'] = ['new']
            self.save_tracking_ids()
        return True
    return False

inquiry.is_auto_reply_needed = types.MethodType(is_auto_reply_needed, inquiry)

In [ ]:
def send_message(self, message, attach=None):
    chat_form = self.browser.find_element_by_css_selector('form.reply-wrapper div.holder')
    chat_form.click()

    uploading_failed = False
    if attach:
        file_input = self.browser.find_element_by_css_selector('input#attachs')
        file_input.send_keys(attach)

        css = 'div.next-upload-list-item:last-child'
        div = WebDriverWait(self.browser, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css)))
        try:
            WebDriverWait(self.browser, 150).until(
                element_has_css_class(div, 'next-upload-list-item-done'))
        except TimeoutException:
            print('time out, cancel uploading')
            div.find_element_by_css_selector('i.next-icon-close').click()
            uploading_failed = True

    if uploading_failed:
        return False

    # if uploading_failed or attach is None:
    #     doc = pq(message)
    #     doc.find('span.attach').remove()
    #     message = doc.outer_html()

    WebDriverWait(self.browser, 15).until(
        EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, '#inquiry-content_ifr')))
    
#     body = self.browser.find_element_by_tag_name('body')
#     self.click(body)

    js = self.reply_js_template.format(message=message)
    self.browser.execute_script(js)
    
    time.sleep(1)
    
    body = self.browser.find_element_by_tag_name('body')
    self.click(body)
    
    self.browser.switch_to.default_content()

    btn_send = self.browser.find_element_by_css_selector('form.reply-wrapper button.send')
    btn_send.click()

    time.sleep(1)
    return True

inquiry.send_message = types.MethodType(send_message, inquiry)

In [ ]:
def reply(self, enquiry):
    try:
        self.open_inquiry_in_new_tab(enquiry)
        conversation = self.get_conversation()

        messages = conversation['messages']
        buyer = conversation['buyer']

        last_buyer_messages = []
        is_replied = False
        reply_count = 0
        for msg in messages:
            if msg['position'] == 'left':
                last_buyer_messages.append(msg)
                is_replied = False
            elif msg['position'] == 'right' and 'Seller Assistant' not in msg['user']:
                last_buyer_messages = []
                is_replied = True
                reply_count += 1
    
        is_replied = False
        # check if email address is already exists in conversation
        text = ''
        for msg in last_buyer_messages:
            text = text + '\n\n ' + msg['content']
        emails = re.findall('([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', text)

        if buyer['email']:
            emails.append(buyer['email'])

        eid = enquiry['id']
        tracking = self.tracking_ids[eid]
        last_status = tracking['status'][-1]
        greetings = '<p>Pleased to hear from you.</p><br>'
        if last_status == 'new':
            if emails:
                mime_message = Email.message_of_product_catalog(self.market, self.account, buyer['name'])
                if Email.send(self.account, emails, mime_message):
                    params = {'buyer': buyer['name'], 'greetings': greetings, 'email': ','.join(emails), 'sender': self.lname, 'whatsapp': self.mobile}
                    message = self.reply_templates['notify_catalog_was_sent'].substitute(params)
                    if self.send_message(message, self.catalog):
                        tracking['status'].append('catalog was sent by email')
                        self.save_tracking_ids()
                    else:
                        tracking['status'].append('catalog was sent by email, but reply was failed')
                        self.save_tracking_ids()
            else:
                params = {'buyer': buyer['name'], 'greetings': greetings, 'sender': self.lname, 'whatsapp': self.mobile}
                message = self.reply_templates['notify_first_reply_without_email'].substitute(params)
                if self.send_message(message, self.catalog):
                    tracking['status'].append('catalog was sent directly')
                    self.save_tracking_ids()
        elif 'catalog was sent' in last_status and 'reply was failed' in last_status:
            if is_replied:
                tracking['status'].append('catalog was sent by email')
                self.save_tracking_ids()
            else:
                params = {'buyer': buyer['name'], 'greetings': greetings, 'email': ','.join(emails), 'sender': self.lname, 'whatsapp': self.mobile}
                message = self.reply_templates['notify_catalog_was_sent'].substitute(params)
                if self.send_message(message, self.catalog):
                    tracking['status'].append('catalog was sent by email')
                    self.save_tracking_ids()

        elif 'catalog was sent' in last_status:
            pass
        elif 'wait for email address' in last_status:
            text = ''
            for msg in last_buyer_messages:
                text = text + '\n\n ' + msg['content']
            emails = re.findall('([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', text)
            print(emails)
            if emails:
                tracking['emails'] = emails
                self.save_tracking_ids()
                mime_message = Email.message_of_product_catalog(self.market, self.account, buyer['name'])
                if Email.send(self.account, emails, mime_message):
                    params = {'buyer': buyer['name'], 'greetings': '', 'email': ', '.join(emails),
                              'sender': self.lname, 'whatsapp': self.mobile}
                    message = self.reply_templates['notify_catalog_was_sent'].substitute(params)
                    if self.send_message(message, self.catalog):
                        tracking['status'].append('catalog was sent by email')
                        self.save_tracking_ids()
    except Exception as e:
        self.notify("danger", '回复 询盘 [' + enquiry['id'] + '] 时 发生错误! ' + str(e))
        traceback.print_exc()
    finally:
        self.close_inquiry_and_switch_back()
inquiry.reply = types.MethodType(reply, inquiry)

In [ ]:
inquiry.load_url()
time.sleep(0.1)

In [5]:
inquiries = inquiry.get_inquiries()

In [10]:
enquiry = inquiries[0]
enquiry['is_new']

False

In [ ]:
enquiry = inquiries[0]
enquiry['is_new'] = True
enquiry

In [ ]:
inquiry.tracking_ids = {}
inquiry.is_auto_reply_needed(enquiry)
inquiry.tracking_ids

In [ ]:
inquiry.reply(enquiry)

In [ ]:
inquiry.account

In [ ]:
inquiry.load_reply_templates()
inquiry.reply_templates['notify_first_reply_without_email']

In [9]:
# inquiry.tracking_ids['11947313067']['status'] = ['new']
inquiry.tracking_ids

{'100765264623': {'datetime': DateTime(2018, 10, 13, 12, 9, 3, tzinfo=Timezone('+08:00')),
  'status': ['new', 'catalog was sent by email']}}

In [8]:
inquiry.check()

E:\glitter\Eyelashes_config [] inquiry_tracking_ids_Ada.json
E:\glitter\Eyelashes_config\inquiry_tracking_ids_Ada.json was deserialized!
enquiry 100764430933 reply is needed
danger 回复 询盘 [100764430933] 时 发生错误! '100764430933'


Traceback (most recent call last):
  File "E:\workspace\python\Market Management\libs\alibaba\inquiry.py", line 329, in reply
    tracking = self.tracking_ids[eid]
KeyError: '100764430933'


In [ ]:
inquiry.check()

In [ ]:
inquiry.tracking_ids

In [ ]:
inquiry.close_inquiry_and_switch_back()

In [ ]:
inquiry.load_find_catalog()

In [ ]:
inquiry.catalog

In [ ]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
inquiry = Inquiry(market)

In [ ]:
emails = ['changshu.qd@gmail.com', 'odingdongo@hotmail.com']
mime_message = Email.message_of_product_catalog(inquiry.market, inquiry.account, 'prestl bernhard')
Email.send(inquiry.account, emails, mime_message)

In [ ]:
inquiry.account

In [ ]:
enquiry = inquiries[8]
enquiry

In [ ]:
enquiry = inquiries[8]
inquiry.open_inquiry_in_new_tab(enquiry)
conversation = inquiry.get_conversation()
inquiry.close_inquiry_and_switch_back()
messages = conversation['messages']
buyer = conversation['buyer']
messages

In [ ]:
enquiry = inquiries[8]
enquiry

In [ ]:
# inquiry.load_tracking_ids()
inquiry.tracking_ids

In [ ]:
inquiry.close_inquiry_and_switch_back()

In [ ]:
# inquiry.close_inquiry_and_switch_back()
# inquiry.tracking_ids['11947313067']['status'] = ['new']
enquiry = inquiries[0]
inquiry.reply(enquiry)

In [ ]:
inquiry.tracking_ids

In [ ]:
def try_login_webww():
    btn_login = WebDriverWait(inquiry.browser, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#webatm2-iconbar')))
    btn_login.click()

In [ ]:
panel = inquiry.browser.find_element_by_css_selector('#webatm2-panel')

In [ ]:
inquiries[11]

In [ ]:
re.findall('([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', text)

In [ ]:
inquiry.browser.save_screenshot("screenshot.png")

In [ ]:
text = 'Inquiry ID：100708962003'

In [ ]:
text = '询价单号：100708962003'

In [ ]:
re.search('(\d+)', text).group(1)

In [ ]:
text

In [ ]:
msg = 'ldskfjsl'
js = "document.querySelector('.webatm2-window-body .webatm2-editor-body').innerHTML = `{message}`;"
js.format(message=msg)

In [ ]:
from string import Template

In [ ]:
params = {'salutation': 'chang', 'link': 'link', 'sender': 'shu'}
templ = Template('$salutation, lsdkfjlskf. $sender')
msg = templ.substitute(params)